In [1]:
 '''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [3]:
NUM_VERTIHUBS = 2000 #dependent on trip_data
FLIGHT_SPEED = 60 #m/s

In [4]:
'''
Trip data formatting
'''
trip_data = pd.read_csv('data/Realtime/OpsLimits/trips_2000.csv')
trip_data.drop(trip_data[trip_data.OriginVertihub == trip_data.DestinationVertihub].index, inplace=True) #drop trips where the origin and destination are the same vertiport, reduced from 4801 trips to 1692 with 10 vertiports
trip_data

,Time,OriginVertihub,OriginVertiport,DestinationVertihub,DestinationVertiport
0,0,333,922,730,476
1,0,1986,459,1800,441
2,0,1540,315,1658,1891
3,0,455,360,1448,1591
4,0,1125,686,1642,437
...,...,...,...,...,...
4796,0,1956,1890,1305,188
4797,0,1316,1243,422,1525
4798,0,1916,516,1549,1797
4799,0,1848,868,366,1044


In [5]:
vertiports_data = pd.read_csv('data/Realtime/OpsLimits/vertiports_2000.csv')
vertiports_data

,vertiport,long,lat,vertihub
0,0,-88.077616,41.691089,1750
1,1,-88.335512,42.284315,136
2,2,-87.590294,41.736803,1962
3,3,-87.867464,42.013719,1767
4,4,-89.143889,42.128561,372
...,...,...,...,...
1995,1995,-87.906844,41.775018,1513
1996,1996,-87.897864,42.095067,1900
1997,1997,-87.568446,41.593811,248
1998,1998,-87.801232,41.536060,360


In [6]:
'''
vertiport statistics
'''
vertiports_per_vertihub = [0 for i in range(NUM_VERTIHUBS)]
for index, vertiport in vertiports_data.iterrows():
    vertiports_per_vertihub[int(vertiport['vertihub'])] += 1
assert(sum(vertiports_per_vertihub) == vertiports_data.shape[0])
vertiports_per_vertihub



[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [7]:
from math import cos, asin, sqrt, pi

# def dist(x1, y1, x2, y2):
#     return math.sqrt((y2-y1)**(y2-y1) + (x2-x1)**(x2-x1))
def ll_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...   
    

In [8]:
'''
trip statistics
'''

arrivals_per_vertihub = [[] for i in range(NUM_VERTIHUBS)]
flight_popularity = dict()
# get the trip arrival time per vertihub
for index, trip in trip_data.iterrows():
    #calculate time of arrival (begin at trip.time), calculated between vertiports
    origin_port = vertiports_data.iloc[trip['OriginVertiport']]
    destination_port = vertiports_data.iloc[trip['DestinationVertiport']]
    if((origin_port['vertiport'],destination_port['vertiport']) not in flight_popularity.keys()):
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] = 1
    else:
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] += 1
    #trip distance in kilometers
    dist = ll_distance(origin_port['lat'], origin_port['long'], destination_port['lat'], destination_port['long'])
    travel_time = (dist*1000)/FLIGHT_SPEED # (km * 1000)/(m/s) -> seconds
    arrival_time = trip['Time'] + travel_time #takeoff time + travel_time = arrival time
    #add arrival time to vertihub
    arrivals_per_vertihub[trip['DestinationVertihub']].append(arrival_time)

# for each list of vertihub, calculate the minimum, average, and maximum distance between arrivals
differences_per_vertihub = []
# simulated_arrivals
for arrivals_list in arrivals_per_vertihub:
    sorted_arrivals = sorted(arrivals_list)
    differences = []
    if(len(sorted_arrivals) <= 1):
        continue
    for index in range(len(sorted_arrivals)-1):
        differences.append(sorted_arrivals[index+1] - sorted_arrivals[index])
    print(sorted_arrivals)
    differences_per_vertihub.append(sorted(differences))

for index, differences in enumerate(differences_per_vertihub):
    print('vertihub ' + str(index) + ' has minimum arrival difference of ' + str(differences[0]))
    print('vertihub ' + str(index) + ' has average arrival difference of ' + str(sum(differences)/len(differences)))
    print('vertihub ' + str(index) + ' has maximum arrival difference of ' + str(differences[len(differences)-1]))
    print('vertihub ' + str(index) + ' has ' + str(len(differences)) + ' trips')
    print('')
   

formatted_flight_popularity = [(flight_popularity[key], key) for key in flight_popularity.keys()]
print('Most popular flight path ' + str((sorted(formatted_flight_popularity))[len(formatted_flight_popularity)-1]))
print('Total flight paths ' + str(len(formatted_flight_popularity)))
avg = sum([num for num, thing in formatted_flight_popularity])/len(formatted_flight_popularity)
print('Mean requests per flight path ' + str(avg))
    
    

[173.47282012515464, 178.33722626054922, 355.1495949064962, 455.3729720541311, 553.9197011550015]
[191.35644772735364, 299.9300365674082, 322.6216278450907, 371.5709795882274]
[75.2816792271121, 275.8197132466165, 302.854622480662]
[161.2812483343683, 714.2344266215327]
[218.46511014712968, 218.46511014712968, 222.39119861211893, 324.81496726493714, 618.8027988166561]
[159.89263007505204, 313.79692685748284, 442.46683066404086]
[125.46632807200476, 336.7036773737968, 414.6178681992782, 572.6941504711026, 579.1448476702657, 590.7242643571806, 691.1168251852895, 707.9342013136217]
[55.148145487815434, 358.8307315554664, 405.0644012221673, 494.55444027758824]
[25.868365430168282, 51.21634776504293, 73.67710211533755, 103.98156450502425, 134.21607348495982, 162.88392486840172, 179.7011869206105, 239.31795594434925, 245.05952515559392, 274.2302144044158, 307.50456966235225]
[80.91989327765964, 113.14978744321337, 170.17229537113977]
[151.2714705858411, 257.75931297689095, 397.3776626545058]

vertihub 99 has average arrival difference of 241.48306015496672
vertihub 99 has maximum arrival difference of 241.48306015496672
vertihub 99 has 1 trips

vertihub 100 has minimum arrival difference of 0.0
vertihub 100 has average arrival difference of 40.15179002094513
vertihub 100 has maximum arrival difference of 64.94450922024674
vertihub 100 has 4 trips

vertihub 101 has minimum arrival difference of 5.253947907995325
vertihub 101 has average arrival difference of 49.906454947908735
vertihub 101 has maximum arrival difference of 158.022496162874
vertihub 101 has 7 trips

vertihub 102 has minimum arrival difference of 330.16901246395213
vertihub 102 has average arrival difference of 448.3207239729451
vertihub 102 has maximum arrival difference of 566.4724354819381
vertihub 102 has 2 trips

vertihub 103 has minimum arrival difference of 194.62434345594568
vertihub 103 has average arrival difference of 194.62434345594568
vertihub 103 has maximum arrival difference of 194.624343455945


vertihub 548 has minimum arrival difference of 103.97555702448119
vertihub 548 has average arrival difference of 114.57839654308228
vertihub 548 has maximum arrival difference of 125.18123606168336
vertihub 548 has 2 trips

vertihub 549 has minimum arrival difference of 10.40259163186505
vertihub 549 has average arrival difference of 30.89964119709511
vertihub 549 has maximum arrival difference of 56.04973269543814
vertihub 549 has 3 trips

vertihub 550 has minimum arrival difference of 0.0
vertihub 550 has average arrival difference of 195.9304568593045
vertihub 550 has maximum arrival difference of 620.4013763382684
vertihub 550 has 4 trips

vertihub 551 has minimum arrival difference of 23.693549373877886
vertihub 551 has average arrival difference of 23.693549373877886
vertihub 551 has maximum arrival difference of 23.693549373877886
vertihub 551 has 1 trips

vertihub 552 has minimum arrival difference of 5.921371187754062
vertihub 552 has average arrival difference of 77.53426853